In [1]:
import pandas as pd
import numpy as np

## Import datasets

In [2]:
diabetes = pd.read_csv('dataset_diabetes/diabetic_data.csv')
train = pd.read_csv('dataset_diabetes/train_multi.csv')
test = pd.read_csv('dataset_diabetes/test_multi.csv')

Combine train and test datasets

In [3]:
train['type'] = 'train'
test['type'] = 'test'

In [4]:
diabetes = train.append(test)

## Preview of two datasets

In [5]:
pd.options.display.max_columns = 55

In [6]:
diabetes.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted,type
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,NaN,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,NaN,NaN,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,train
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,NaN,NaN,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30,train
2,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,NaN,NaN,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO,train
3,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,NaN,NaN,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO,train
4,35754,82637451,Caucasian,Male,[50-60),NaN,2,1,2,3,NaN,NaN,31,6,16,0,0,0,414,411,250,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30,train


In [7]:
print(diabetes.shape)

(101766, 51)


### Split id_mapping into 3 unique dataframes

In [8]:
f = open('dataset_diabetes/IDs_mapping.csv', 'r')    # 'r' for read
lines = f.readlines()
f.close()

In [9]:
indices = [i for i, x in enumerate(lines) if x == ',\n']

In [10]:
indices

[9, 41]

In [11]:
admission_type = pd.read_csv('dataset_diabetes/IDs_mapping.csv', skiprows=list(range(9,len(lines))))

rows_skip = list(range(0,10)) + list(range(41, len(lines)))
discharge_disposition = pd.read_csv('dataset_diabetes/IDs_mapping.csv', skiprows=rows_skip)

admission_source = pd.read_csv('dataset_diabetes/IDs_mapping.csv', skiprows=list(range(42)))

### Combine values in all three ID tables

admission_type

In [12]:
## make 'Not Available', 'NaN', and 'Not Mapped' all 'NA'
for i in range(len(admission_type)):
    x = admission_type['description'][i]
    if (x == 'Not Available' or x == 'Not Mapped'):
        admission_type['description'][i] = np.nan

/home/anisha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


discharge_disposition

In [13]:
## make 'Unknown/Invalid', 'NaN', and 'Not Mapped' all 'NA'
for i in range(len(discharge_disposition)):
    x = discharge_disposition['description'][i]
    if (x == 'Unknown/Invalid' or x == 'Not Mapped'):
        discharge_disposition['description'][i] = np.nan
    elif(pd.isna(x)):
        continue
    elif ('Expired' in x):
        discharge_disposition['description'][i] = 'Expired'

/home/anisha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/anisha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


admission_source

In [14]:
## make 'Not Available', 'NaN', 'Unknown/Invalid' and 'Not Mapped' all 'NA'
for i in range(len(admission_source)):
    x = admission_source['description'][i]
    if(pd.isna(x)):
        continue
    elif (x.strip() == 'Not Available' or x.strip() == 'Not Mapped' or x.strip() == 'Unknown/Invalid'):
        admission_source['description'][i] = np.nan

/home/anisha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


## Merge Datasets

In [15]:
ds = diabetes.merge(admission_type, how = 'left', on='admission_type_id')
ds2 = ds.merge(discharge_disposition, how = 'left', on = 'discharge_disposition_id')
full_dataset = ds2.merge(admission_source, how = 'left', on = 'admission_source_id')
full_dataset.rename(columns={"description_x": "admissionType",
                             "description_y": "dischargeDisposition",
                             "description" : "admissionSource"}, inplace = True)

In [16]:
full_dataset.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted,type,admissionType,dischargeDisposition,admissionSource
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,NaN,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,NaN,NaN,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,train,NaN,NaN,Physician Referral
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,NaN,NaN,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30,train,Emergency,Discharged to home,Emergency Room
2,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,NaN,NaN,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO,train,Emergency,Discharged to home,Emergency Room
3,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,NaN,NaN,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO,train,Emergency,Discharged to home,Emergency Room
4,35754,82637451,Caucasian,Male,[50-60),NaN,2,1,2,3,NaN,NaN,31,6,16,0,0,0,414,411,250,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30,train,Urgent,Discharged to home,Clinic Referral


#### Remove admission_type_id, discharge_disposition_id, and admission_source_id

In [17]:
full_dataset.drop(['admission_type_id', 'discharge_disposition_id', 'admission_source_id'], axis = 1, inplace = True)

In [18]:
full_dataset.shape

(101766, 51)

### Make Response Variable - readmitted_30
#### Make readmitted_30 column
Keep column 'readmitted' because >30 could be an indicator <- not doing this

In [19]:
np.unique(full_dataset.readmitted.values)

array(['<30', '>30', 'NO'], dtype=object)

In [20]:
full_dataset['readmitted'] = ['Yes' if x == '<30' else 'No' for x in full_dataset['readmitted']]

### Remove unnecessary columns

#### Remove weight from dataset (97% missing)

In [21]:
full_dataset.drop(['weight'], axis = 1, inplace = True)

#### If only one value in column, remove column
This includes only citoglipton and examide
#### If less than 1000 (1%) not 'No' in column, remove column
Can change threshold number, calculated columns below

In [22]:
nos = []
for i in range(18,43):
    col = full_dataset.columns[i]
    numNotNo = len(full_dataset[full_dataset[col] != 'No'])
    if (numNotNo < 1000):
        nos.append(col)

In [23]:
nos

['nateglinide',
 'chlorpropamide',
 'acetohexamide',
 'tolbutamide',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'examide',
 'citoglipton',
 'glyburide.metformin',
 'glipizide.metformin',
 'glimepiride.pioglitazone',
 'metformin.rosiglitazone',
 'metformin.pioglitazone']

In [24]:
full_dataset.drop(nos, axis = 1, inplace = True)

In [25]:
full_dataset.shape

(101766, 35)

### Make columns for diagnosis categories
#### Convert icd9 codes to to categories
This is under the diag_1,2,3 column

In [26]:
def code_to_category(icd9code):
    try:
        icd9code = float(icd9code)
        if (icd9code >= 390 and icd9code < 460) or icd9code == 785:
            return 'Circulatory'
        if (icd9code >= 460 and icd9code < 520) or icd9code == 786:
            return 'Respiratory'
        if (icd9code >= 520 and icd9code < 580) or icd9code == 787:
            return 'Digestive'
        if (icd9code >= 250 and icd9code < 251):
            return 'Diabetes'
        if (icd9code >= 800 and icd9code < 1000):
            return 'Injury'
        if (icd9code >= 710 and icd9code < 740):
            return 'Musculoskeletal'
        if (icd9code >= 580 and icd9code < 630) or icd9code == 788:
            return 'Genitourinary'
        if (icd9code >= 140 and icd9code < 240) or icd9code == 780 or icd9code == 781 or icd9code == 784 or (icd9code >= 790 and icd9code < 800):
            return 'Neoplasms'
        if (icd9code >= 240 and icd9code < 250) or (icd9code >= 251 and icd9code < 280) or (icd9code >= 680 and icd9code < 710) or icd9code == 782:
            return 'Neoplasms'
        else:
            return 'Other'
    except:
        if (icd9code == '?'):
            return '?'
        return 'Other'

In [27]:
full_dataset['diag_1_cat'] = [code_to_category(x) for x in full_dataset['diag_1']]
full_dataset['diag_2_cat'] = [code_to_category(x) for x in full_dataset['diag_2']]
full_dataset['diag_3_cat'] = [code_to_category(x) for x in full_dataset['diag_3']]

In [28]:
full_dataset.drop(['diag_1', 'diag_2','diag_3'], axis = 1, inplace = True)

### Remove patients if have 'expired'
This remove 1,652 rows

In [29]:
expired = full_dataset[full_dataset['dischargeDisposition'] == 'Expired'].index
full_dataset.drop(expired, axis = 0, inplace = True)

In [30]:
full_dataset.shape

(100114, 35)

### Make only 3 values for medicines: No, Steady, Change
If column has 'Up' or 'Down', then change these observations to 'Change'.
Remove 'change' column, as this is indicated in medicine's columns.

In [31]:
for i in range(17,25):
    col = full_dataset.columns[i]
    full_dataset[col] = ['Change' if (x == 'Down' or x == 'Up') else x for x in full_dataset[col]]

What do we do about 'max_glu_serum' and 'A1Cresult'?

### Remove 'change' column.

In [32]:
full_dataset.drop('change', axis = 1, inplace = True)

## Dummify

In [33]:
full_dataset

,encounter_id,patient_nbr,race,gender,age,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,insulin,diabetesMed,readmitted,type,admissionType,dischargeDisposition,admissionSource,diag_1_cat,diag_2_cat,diag_3_cat
0,2278392,8222157,Caucasian,Female,[0-10),1,NaN,Pediatrics-Endocrinology,41,0,1,0,0,0,1,None,None,No,No,No,No,No,No,No,No,No,No,train,NaN,NaN,Physician Referral,Diabetes,Other,Other
1,149190,55629189,Caucasian,Female,[10-20),3,NaN,NaN,59,0,18,0,0,0,9,None,None,No,No,No,No,No,No,No,Change,Yes,No,train,Emergency,Discharged to home,Emergency Room,Neoplasms,Diabetes,Neoplasms
2,500364,82442376,Caucasian,Male,[30-40),2,NaN,NaN,44,1,16,0,0,0,7,None,None,No,No,No,No,No,No,No,Change,Yes,No,train,Emergency,Discharged to home,Emergency Room,Other,Diabetes,Circulatory
3,16680,42519267,Caucasian,Male,[40-50),1,NaN,NaN,51,0,8,0,0,0,5,None,None,No,No,No,Steady,No,No,No,Steady,Yes,No,train,Emergency,Discharged to home,Emergency Room,Neoplasms,Neoplasms,Diabetes
4,35754,82637451,Caucasian,Male,[50-60),3,NaN,NaN,31,6,16,0,0,0,9,None,None,No,No,No,No,No,No,No,Steady,Yes,No,train,Urgent,Discharged to home,Clinic Referral,Circulatory,Circulatory,Diabetes
5,55842,84259809,Caucasian,Male,[60-70),4,NaN,NaN,70,1,21,0,0,0,7,None,None,Steady,No,Steady,No,No,No,No,Steady,Yes,No,train,Elective,Discharged to home,Clinic Referral,Circulatory,Circulatory,Other
6,63768,114882984,Caucasian,Male,[70-80),5,NaN,NaN,73,0,12,0,0,0,8,None,None,No,No,No,No,Steady,No,No,No,Yes,No,train,Emergency,Discharged to home,Emergency Room,Circulatory,Respiratory,Diabetes
7,15738,63555939,Caucasian,Female,[90-100),12,NaN,InternalMedicine,33,3,18,0,0,0,8,None,None,No,No,No,No,No,No,Steady,Steady,Yes,No,train,Elective,Discharged/transferred to SNF,Transfer from a hospital,Circulatory,Neoplasms,Respiratory
8,36900,77391171,AfricanAmerican,Male,[60-70),7,NaN,NaN,62,0,11,0,0,0,7,None,None,No,No,No,No,Change,No,No,Steady,Yes,Yes,train,Urgent,Discharged to home,Transfer from a hospital,Neoplasms,Other,Neoplasms
9,42570,77586282,Caucasian,Male,[80-90),10,NaN,Family/GeneralPractice,55,1,31,0,0,0,8,None,None,No,No,No,No,No,No,No,Steady,Yes,No,train,Emergency,Discharged/transferred to home with home healt...,Emergency Room,Circulatory,Circulatory,Circulatory


In [34]:
df = full_dataset.copy()

numerical = list(set(df.mean().index))
categorical = list(set(df.columns) - set(numerical))

In [35]:
# Create dummies
one_hot_data = pd.get_dummies(df[categorical],drop_first=True)

#Drop categorical
df = full_dataset.drop(categorical, axis=1)

# Merge with one hot
df = df.join(one_hot_data)
df.shape

(100114, 198)

In [36]:
df.columns.tolist()

['encounter_id',
 'patient_nbr',
 'time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses',
 'age_[10-20)',
 'age_[20-30)',
 'age_[30-40)',
 'age_[40-50)',
 'age_[50-60)',
 'age_[60-70)',
 'age_[70-80)',
 'age_[80-90)',
 'age_[90-100)',
 'glyburide_No',
 'glyburide_Steady',
 'medical_specialty_Anesthesiology',
 'medical_specialty_Anesthesiology-Pediatric',
 'medical_specialty_Cardiology',
 'medical_specialty_Cardiology-Pediatric',
 'medical_specialty_DCPTEAM',
 'medical_specialty_Dentistry',
 'medical_specialty_Dermatology',
 'medical_specialty_Emergency/Trauma',
 'medical_specialty_Endocrinology',
 'medical_specialty_Endocrinology-Metabolism',
 'medical_specialty_Family/GeneralPractice',
 'medical_specialty_Gastroenterology',
 'medical_specialty_Gynecology',
 'medical_specialty_Hematology',
 'medical_specialty_Hematology/Oncology',
 'medical_specialty_Hospitalist',
 'medical_sp

### Columns to drop
drop columns that occur less than 1000 times

In [37]:
lessFreq = []
for i in range(10, len(df.columns)):
    col = df.columns[i]
    try:
        if(df[col].value_counts()[1] < 1000):
            lessFreq.append(col)
    except:
        continue

In [38]:
# lessFreq

In [39]:
df.drop(lessFreq, axis=1, inplace=True)

In [40]:
df.drop(['patient_nbr'], axis=1, inplace=True)

In [41]:
df[df['type_train'] == 0].shape

(20030, 100)

## Split test and train datasets into new csvs

In [42]:
new_train = df.loc[df['type_train']==1,:]
new_test = df.loc[df['type_train']==0,:].copy()
new_train.drop('type_train', axis=1, inplace=True)
new_test.drop('type_train', axis=1, inplace=True)

/home/anisha/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [43]:
# Save new test and train sets
new_test.to_csv('./dataset_diabetes/new_test.csv')
new_train.to_csv('./dataset_diabetes/new_train.csv')

In [44]:
full_dataset.drop('type', axis = 1, inplace=True)
full_dataset.to_csv('./dataset_diabetes/new_dataset.csv')

#### Impute Missing Values
Values missing from:
- race
- payer_code
- medical_speciality
- diag_1_cat
- diag_2_cat
- diag_3_cat